In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [6]:
!pip install datasets

In [7]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

In [25]:
import pandas as pd
df = pd.read_csv('TrainingData.tsv', sep='\t')
df.columns =['id1', 'id2', 'label', 'text']

In [26]:
df2 = pd.read_csv('DevData.tsv', sep='\t')
df2.columns =['id1', 'id2', 'label', 'text']

In [27]:
df3 = pd.read_csv('TestData.tsv', sep='\t', on_bad_lines='skip')
df3.columns =['id1', 'id2', 'label', 'text']

In [30]:
x_train, y_train = df['text'],df['label']
x_valid, y_valid = df2['text'],df2['label']
x_test, y_test = df3['text'],df3['label']


str_to_idx = {
    'negative':0,
    'positive':1,
    'neutral':2
}
y_train2 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_train))
y_train2 = list(map(lambda x: x.replace('objective', 'neutral'), y_train2))
y_train = [str_to_idx[c] for c in y_train2]

y_valid2 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_valid))
y_valid2 = list(map(lambda x: x.replace('objective', 'neutral'), y_valid2))
y_valid = [str_to_idx[c] for c in y_valid2]


y_test3 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_test))
y_test3 = list(map(lambda x: x.replace('objective', 'neutral'), y_test3))
y_test = [str_to_idx[c] for c in y_test3]

In [31]:
len(y_train)

8001

In [32]:
df = pd.DataFrame(list(zip(x_train, y_train)), columns = ['text', 'label'])
df2 = pd.DataFrame(list(zip(x_valid, y_valid)), columns = ['text', 'label'])
df3 = pd.DataFrame(list(zip(x_test, y_test)), columns = ['text', 'label'])

In [33]:
hg_dataset_train = Dataset(pa.Table.from_pandas(df))
hg_dataset_valid = Dataset(pa.Table.from_pandas(df2))
hg_dataset_test = Dataset(pa.Table.from_pandas(df3))

In [34]:
import datasets
tweets = datasets.DatasetDict({"train":hg_dataset_train,"valid":hg_dataset_valid,"test":hg_dataset_test})


In [35]:
tweets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8001
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1380
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3136
    })
})

In [36]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [37]:
tweets_encoded = tweets.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/8001 [00:00<?, ? examples/s]

Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

Map:   0%|          | 0/3136 [00:00<?, ? examples/s]

In [38]:
tweets_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8001
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1380
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3136
    })
})

In [51]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

tweets_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 4

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(tweets_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(tweets_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [52]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[  101,  1030, 25933, ...,     0,     0,     0],
       [  101,  2066,  1010, ...,     0,     0,     0],
       [  101,  3427, 18437, ...,     0,     0,     0],
       [  101,  1030, 22822, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([1 1 2 2], shape=(4,), dtype=int64)


In [53]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [54]:
classifier = BERTForClassification(model, num_classes=3)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [55]:
train_dataset

<_ParallelMapDataset element_spec=({'input_ids': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [56]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
2001/2001 [==============================] - 1051s 490ms/step - loss: 0.7872 - accuracy: 0.6372
Epoch 2/3
2001/2001 [==============================] - 981s 490ms/step - loss: 0.4608 - accuracy: 0.8150
Epoch 3/3
2001/2001 [==============================] - 979s 489ms/step - loss: 0.2797 - accuracy: 0.8921


In [57]:
classifier.evaluate(test_dataset)

784/784 [==============================] - 141s 176ms/step - loss: 0.6041 - accuracy: 0.7503


[0.6041151285171509, 0.7503188848495483]

In [58]:
import pickle
pickle.dump(classifier, open("tweet-classifier-bert.pkl", "wb"))


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
